In [1]:
import os, csv
import scipy, numpy as np, pandas as pd, time
from scipy import sparse
import pyBigWig, prep_metadata_labels

# Human chromosome names
chr_IDs = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
_data_dir = '../../examples/data/encode-tfbs_v1.0/'

# Prep metadata df and metadata/label array
- Metadata df contains 6400bp (window_size/2) prediction windows across the genome. Each gets a 128-bit prediction from the model.
- We store the ones that aren't fully unbound, and write these to bigwigs representing genome-wide labels.

In [4]:
prep_metadata_labels.write_label_bigwigs()

83.30138063430786
H1-hESC 100.73247504234314
HCT116 106.4023334980011
HeLa-S3 111.88021206855774
HepG2 117.56940197944641
K562 126.93423342704773
A549 138.21517205238342
GM12878 148.77391648292542
150.62964010238647
213.72714066505432


- Then read from the bigwigs to generate metadata for the bound sites.

In [7]:
stride = 6400
itime = time.time()
mdf_posamb = pd.read_csv(
    _sorted_dir, 
    sep='\t', header=None, index_col=None, names=['chr', 'start', 'stop', 'y', 'celltype']
)
celltype_mdta = []
celltype_labels = []
_train_celltypes = ['H1-hESC', 'HCT116', 'HeLa-S3', 'HepG2', 'K562']
_val_celltype = ['A549']
_test_celltype = ['GM12878']
_all_celltypes = _train_celltypes + _val_celltype + _test_celltype

for ct in _all_celltypes:
    ct_labels_bw_path = _data_dir + "labels/MAX/MAX_{}.bigwig".format(ct)
    df = mdf_posamb[mdf_posamb['celltype'] == ct]
    df['window_start'] = stride*(df['start'] // stride)
    uniq_windows = np.unique(["{}:{}".format(x[0], x[1]) for x in zip(df['chr'], df['window_start'])])
    df_construction = []
    mdta_labels = []
    
    bw = pyBigWig.open(ct_labels_bw_path)
    num_reps = 0
    for u in uniq_windows:
        u_chr = u.split(':')[0]
        u_start = int(u.split(':')[1])
        u_end = u_start + stride
        x = np.nan_to_num(bw.values(u_chr, u_start, u_end, numpy=True))
        df_construction.append((u_chr, u_start, u_end))
        mdta_labels.append(x[np.arange(0, len(x), 50)])
        num_reps = num_reps + 1
    celltype_mdta_df = pd.DataFrame(df_construction, columns=['chr', 'start', 'stop'])
    celltype_mdta_df.insert(len(celltype_mdta_df.columns), 'celltype', ct)
    celltype_mdta.append(celltype_mdta_df)
    celltype_labels.append(np.stack(mdta_labels))
    print(ct, time.time() - itime)
    bw.close()
    # break
print(time.time() - itime)
# _metadata_df

pd.concat(celltype_mdta).to_csv(
    _data_dir + 'labels/MAX/metadata_df.bed', 
    sep='\t', header=False, index=False
)
np.save(_data_dir + 'labels/MAX/metadata_y.npy', np.vstack(celltype_labels))
print(time.time() - itime)

<ipython-input-7-fc078c66e9a6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['window_start'] = stride*(df['start'] // stride)


A549 63.97912120819092
GM12878 103.89278292655945
H1-hESC 182.84059262275696
HCT116 243.95744681358337
HeLa-S3 303.7187397480011
HepG2 375.8099205493927
K562 456.08897161483765
456.0923991203308
462.8749210834503


## Add the all-unbound sites

In [3]:
stride = 6400
itime = time.time()
mdf_posamb = pd.read_csv(
    _data_dir + 'labels/MAX/MAX_posamb.sorted.bed', 
    sep='\t', header=None, index_col=None, names=['chr', 'start', 'stop', 'y', 'celltype']
)
celltype_mdta = []
celltype_labels = []
_train_celltypes = ['H1-hESC', 'HCT116', 'HeLa-S3', 'HepG2', 'K562']
_val_celltype = ['A549']
_test_celltype = ['GM12878']
_all_celltypes = _train_celltypes + _val_celltype + _test_celltype

for ct in _all_celltypes:
    ct_labels_bw_path = _data_dir + "labels/MAX/MAX_{}.bigwig".format(ct)
    df_construction = []
    mdta_labels = []
    bw = pyBigWig.open(ct_labels_bw_path)
    for chrID in bw.chroms():
        chromsize = bw.chroms()[chrID]
        # Iterate over windows
        for startc in np.arange(0, chromsize-(2*stride), stride):
            u_end = startc + stride
            if u_end > chromsize:
                break
            x = np.nan_to_num(bw.values(chrID, startc, u_end, numpy=True))
            df_construction.append((chrID, startc, u_end))
            mdta_labels.append(x[np.arange(0, len(x), 50)])
        print(ct, chrID, time.time() - itime)
    celltype_mdta_df = pd.DataFrame(df_construction, columns=['chr', 'start', 'stop'])
    celltype_mdta_df.insert(len(celltype_mdta_df.columns), 'celltype', ct)
    celltype_mdta.append(celltype_mdta_df)
    celltype_labels.append(np.stack(mdta_labels))
    print(ct, time.time() - itime)
    bw.close()
    # break
print(time.time() - itime)

H1-hESC chr1 20.20913076400757
H1-hESC chr10 28.008360862731934
H1-hESC chr11 36.41605806350708
H1-hESC chr12 44.10679650306702
H1-hESC chr13 51.19197916984558
H1-hESC chr14 57.02869009971619
H1-hESC chr15 62.31191349029541
H1-hESC chr16 67.81044888496399
H1-hESC chr17 72.55425524711609
H1-hESC chr18 78.06788182258606
H1-hESC chr19 81.6804301738739
H1-hESC chr2 96.18858242034912
H1-hESC chr20 100.57028126716614
H1-hESC chr21 103.48856258392334
H1-hESC chr22 106.65493178367615
H1-hESC chr3 119.19174075126648
H1-hESC chr4 131.20506811141968
H1-hESC chr5 142.5725815296173
H1-hESC chr6 152.76653385162354
H1-hESC chr7 162.29314422607422
H1-hESC chr8 172.217839717865
H1-hESC chr9 180.51852083206177
H1-hESC chrX 189.65529799461365
H1-hESC 190.9764485359192
HCT116 chr1 206.9115183353424
HCT116 chr10 214.90280389785767
HCT116 chr11 223.43896079063416
HCT116 chr12 231.92131686210632
HCT116 chr13 238.30261087417603
HCT116 chr14 244.51456451416016
HCT116 chr15 250.19079542160034
HCT116 chr16 255.5

In [29]:
bw = pyBigWig.open(ct_labels_bw_path)
print(bw.chroms())
bw.close()

{'chr1': 249250621, 'chr10': 135534747, 'chr11': 135006516, 'chr12': 133851895, 'chr13': 115169878, 'chr14': 107349540, 'chr15': 102531392, 'chr16': 90354753, 'chr17': 81195210, 'chr18': 78077248, 'chr19': 59128983, 'chr2': 243199373, 'chr20': 63025520, 'chr21': 48129895, 'chr22': 51304566, 'chr3': 198022430, 'chr4': 191154276, 'chr5': 180915260, 'chr6': 171115067, 'chr7': 159138663, 'chr8': 146364022, 'chr9': 141213431, 'chrX': 155270560}


In [4]:
all_metadata_df = pd.concat(celltype_mdta)
print(time.time() - itime)
all_metadata_df.to_csv(
    _data_dir + 'labels/MAX/all_metadata_df.bed', 
    sep='\t', header=False, index=False
)
print(time.time() - itime)
np.save(_data_dir + 'labels/MAX/all_metadata_y.npy', np.vstack(celltype_labels))
print(time.time() - itime)

4135.834958076477
4144.659387350082
4163.944329023361


In [5]:
all_metadata_df

,chr,start,stop,celltype
0,chr1,0,6400,H1-hESC
1,chr1,6400,12800,H1-hESC
2,chr1,12800,19200,H1-hESC
3,chr1,19200,25600,H1-hESC
4,chr1,25600,32000,H1-hESC
...,...,...,...,...
474383,chrX,155232000,155238400,GM12878
474384,chrX,155238400,155244800,GM12878
474385,chrX,155244800,155251200,GM12878
474386,chrX,155251200,155257600,GM12878
